In [4]:
# import di tutte le librerie che devono essere usate nel progetto
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from collections import defaultdict
from scipy import interp
import numpy as np
import itertools
%matplotlib inline
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_curve
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly
import plotly.plotly as py
from plotly.plotly import plot, iplot
import plotly.graph_objs as go
import warnings
from collections import Counter
import math
from scipy.cluster import hierarchy as hc
import scipy.spatial as scs
import scipy.stats as ss
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, learning_curve, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
import time
from sklearn.pipeline import make_pipeline, Pipeline
from collections import Counter
from prettytable import PrettyTable
from functools import wraps

plotly.tools.set_credentials_file(username='albertobenincasa', api_key='14CCI1wSA0JGaDZhuouL')
warnings.filterwarnings("ignore")

In [54]:
# costanti
PLOTLY_COLORS = ['#4A708B', '#87CEFA']
PLOTLY_OPACITY = 0.7
COLORSCALE_HEATMAP = [         [0.0, '#011f4b'],
                [0.1111111111111111, '#03396c'], 
                [0.2222222222222222, '#005b96'], 
                [0.3333333333333333, '#2171b5'], 
                [0.4444444444444444, '#6497b1'], 
                [0.5555555555555556, '#6baed6'], 
                [0.6666666666666666, '#B0E2FF'], 
                [0.7777777777777778, '#b3cde0'], 
                [0.8888888888888888, '#bdd7e7'], 
                               [1.0, '#BFEFFF']] 
COLOR_PALETTE = sns.color_palette("Blues").as_hex()
RANDOM_SEED = 11

LOGISTIC_REGRESSION_PARAMS = [{
    'clf__solver': ['liblinear'],  # best for small datasets
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], # smaller value, stronger regularization, like svm
    'clf__penalty': ['l2', 'l1']
},
{
    'clf__solver': ['newton-cg', 'lbfgs'], 
    'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l2'] # `newton-cg` and `lbfgs` accept only l2
}]

### INTRODUZIONE

Il dataset oggetto di questa analisi, visualizzabile e scaricabile all'indirizzo https://www.kaggle.com/ronitf/heart-disease-uci, consta di una serie di parametri di paziente affetti o no da disturbi di tipo cardiaco; esso deriva da un database contenente in origine 76 attributi, ma tutti le analisi effettuate si riferiscono ad un sottoinsieme di essi di 14 attributi.

Scendendo più nel dettaglio, apprendiamo che questi 14 attributi specificano diverse caratteristice, di seguito elencate:
* age : età del paziente in anni
* sex : sesso del paziente (1=maschio, 0=femmina)
* cp : tipologia di dolore addominale riscontrata (1=Angina pectoris tipica, 2=Angina pectoris atipica, 3=Dolore non dovuto ad Angina pectoris , 4=Asintomatico)
* trestbps : pressione del sangue a riposo del paziente (in mmHg all'ammissione in ospedale)
* chol : colesterolo del paziente misurato in mg/dl
* fbs : glicemia a digiuno > 120 mg/dl (1=vero; 0=falso) 
* restecg : risutati elettrocardiografici a riposo (0=normale, 1=presenza di anormalità nell'onda ST-T, 2=probabile ipertrofia ventricolo sinistro)  
* thalach : battito cardiaco massimo raggiunto
* exang : angina pectoris indotta da esercizio (1=si, 0=no) 
* oldpeak : sottoslivellamento del tratto ST indotta dall'esercizio, non presente a riposo
* slope : la pendenza durante il picco di esercizio del tratto ST
* ca : numero di vasi maggiori (0-3) colorati per fluoroscopia
* thal : malattia del sangue chiamata talassemia (3 = normale; 6 = difetto fisso; 7 = difetto reversibile)
* target : presenza di malattia (1=si, 0=no)

### CARICAMENTO DEL DATASET E ANALIZI PRELIMINARI

Come prima cosa carichiamo il dataset in una variabile di tipo DataFrame.

In [6]:
# Carico il dataset presente in locale
dataset = pd.read_csv("./dataset/heart.csv")

Il passo successivo consiste nel guardare il dataset appena caricato in modo da capirne il contenuto, le diverse feature disponibili e il tipo di dato presente in quest'ultime.

In [11]:
# Dimensione del dataset
print("Il dataset ha %d record e %d features.\n" % dataset.shape)

# Conteggio del numero di classi per la classificazione
print(f"Ci sono {dataset['target'].unique().size} classi differenti:"f"\n {dataset['target'].unique().tolist()}")

# Conteggio del numero di valori unici per ogni colonna
print(f"\nValori unici per ogni campo: \n{dataset.nunique()}")

Il dataset ha 303 record e 14 features.

Ci sono 2 classi differenti:
 [1, 0]

Valori unici per ogni campo: 
age          41
sex           2
cp            4
trestbps     49
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
target        2
dtype: int64


### PREPROCESSING

Prima di poter passare alla classificazione, è fondamentale una fase di preprocessing dei dati. 
Questo è lo step più importante di tutti, perchè "pulire" i dati prima di "mandarli in pasto" ai diversi algoritmi di classificazione ha un duplice vantaggio: per prima cosa, evitiamo di sovraccaricare la potenza di calcolo della macchina per conti non utili alla classificazione finale, e per seconda, analizzare i dati prima di processarli ci permette di ottenere la classificazione migliore possibile.

Molto utile anche controllare la presenza di eventuali valori nulli: nel caso in cui ci fossero colonne non valorizzate, è bene procedere con l'eliminazione di quest'ultime per ridurre il numero di dati su cui dobbiamo lavorare.

Come possiamo notare, non vi è traccia della presenza di valori nulli nel dataset. 
La strategia alternativa all'eliminazione di intere colonne, particolarmente vantaggiosa solo se tutti i valori (o perlomeno, la maggior parte) sono non definiti è quella di inserire valori di default (come 0) per i record che non hanno valori definiti.

In [5]:
# Controllo la presenza di valori nulli
print(f"\nConteggio dei valori NaN: \n{dataset.isnull().sum()}")


Conteggio dei valori NaN: 
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


La prima, semplice, analisi da fare è quella di vedere che tipo di dati abbiamo.

In [4]:
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Come possiamo notare, tutti i nostri attributi sono di tipo numerico. Nonostante questo, dalla descrizione del dataset sappiamo che alcuni di questi valori corrispondono in realtà ad elementi di tipo categorico. 

Infatti, i campi valorizzati con delle stringhe non permettono un'analisi semplice, 
bla
bla
bla

Vediamo ora come i nostri campioni sono suddivisi nelle diverse classi

In [6]:
y = dataset['target'].value_counts()
print(y)

1    165
0    138
Name: target, dtype: int64


Fortunatamente, il dataset è abbastanza bilanciato: abbiamo circa lo stesso numero di campioni classificati come malati e non malati. Questo è molto importante perchè possiamo assegnare lo stesso peso alle due classi quando passeremo alla fase di classificazione.

Inoltre, possiamo notare come il nostro compito di classificazione sarà di tipo binario: infatti, il nostro obiettivo è riuscire a predire se un paziente potrebbe riscontrare malattie cardiache oppure no. Possiamo ora approfondire alcuni dettagli statistici sul dataset utilizzando il metodo **describe()** sul nostro DataFrame di pandas. L'output mostrerà:
* **count** -> specifica il numero dei record presenti nel dataset 
* **mean** -> specifica la media dell'attributo calcolata per tutti i record
* **std** -> specifica la deviazione standard dell'attributo
* **min** -> specifica il valore minimo dell'attributo
* **25%** -> il 25% dei record ha un valore minore di quello visualizzato (lower percentile)
* **50%** -> il 50% dei record ha un valore minore di quello visualizzato (median percentile)
* **75%** -> il 75% dei record ha un valore minore di quello visualizzato (upper percentile)
* **max** -> specifica il valore massimo dell'attributo

In [7]:
dataset.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [25]:
def create_bar(type, data, col, visible=False):
    """
    Creazione di un istogramma
    
    """
    
    
    if type == "sick":
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
    
    return go.Histogram(
        x = data[col],
        name = type,
        marker = dict(color = c),
        visible = visible,
        opacity = PLOTLY_OPACITY,
    )

def feature_histogram(data, feature, title):
    
    trace1 = create_bar("sick", data[data['target'] == 1], feature, True)
    trace2 = create_bar("healthy", data[data['target'] == 0], feature, True)
    
    data = [trace1, trace2]
    
    layout = dict(
        title=title,
        autosize=True,
        yaxis=dict(
            title='value',
            automargin=True,
        ),
        legend=dict(
            x=0,
            y=1,
        ),
        barmode='group',
        bargap=0.15,
        bargroupgap=0.1
    )
    fig = dict(data=data, layout=layout)

    return py.iplot(fig, filename=title)

feature_histogram(dataset, 'target', 'Distribuzione classi')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~albertobenincasa/0 or inside your plot.ly account where it is named 'Distribuzione classi'


A questo punto possiamo analizzare la distribuzione dei nostri dati usando dei boxplot: un boxplot è un modo standardizzato per visualizzare la distribuzione dei dati sulla base di un riepilogo di cinque numeri (min, lower percentile, median, upper percentile, max); risulta molto utile per visualizzare dati anomali e per controllare se i dati sono ditribuiti in modo simmetrico, quanto sono aggregati.

Le informazioni che possiamo ricavare da un box plot sono:
* **median**(Q2 / 50esimo percentile): il valore medio del dataset
* **first quartile**(Q1 / 25esimo percentile): il valore medio tra il valore più piccolo (non il minimo) e il median del dataset
* **third quartile**(Q3 / 75esimo percentile): il valore medio tra il valore più grande (non il massimo) e il median del dataset
* **interquartile range**(IQR): dal 25esimo al 75esimo percentile
* **outliers**: elementi visualizzati come singoli punti
* **maximum**: Q3 + 1.5xIQR
* **minimum**: Q1 - 1.5xIQR

Chiaramente, ha poco senso visualizzare in questo tipo di grafici quegli attributi che sono binari o che presentano poche opzioni, per cui filtreremo questi dati prima di visualizzare i grafici.

In [23]:
def create_box(type, data, col, visible=False):
    """
    Creazione di una box
    """   
    if type == 'sick':
        c = PLOTLY_COLORS[0]
    else:
        c = PLOTLY_COLORS[1]
        
    return go.Box(
        y = data[col],
        name = type,
        marker = dict(color = c),
        visible = visible,
        opacity = PLOTLY_OPACITY,
    )

sicks = dataset[dataset['target'] == 1]
healthy = dataset[dataset['target'] == 0]
box_features = [col for col in dataset.columns if ((col != 'class') and (dataset[col].nunique() > 5))]

active_index = 0

box_sick = [(create_box('sick', sicks, col, False) if i != active_index
           else create_box('sick', sicks, col, True))
             for i, col in enumerate(box_features)
            ]
box_healthy = [(create_box('healthy', healthy, col, False) if i != active_index
            else create_box('healthy', healthy, col, True))
             for i, col in enumerate(box_features)
            ]

#box_sick = []
#box_healthy = []
#for i, col in enumerate(box_features):
#    if i != active_index:
#        trace1 = create_box('sick', sicks, col, False)
#        trace2 = create_box('healthy', healthy, col, False)
#    else:
#        trace1 = create_box('sick', sicks, col, True)
#        trace2 = create_box('healthy', healthy, col, True)
#    
#    box_sick.append(trace1)
#    box_healthy.append(trace2)

data = box_sick + box_healthy
number_of_features = len(box_features)
steps = []

for i in range(number_of_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * number_of_features],
        label = box_features[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    autosize=True,
    yaxis=dict(
        title='valori',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='box_slider')

commenti sul grafico sopra

introduzione agli istrogrammi di sotto

In [28]:
hist_features = [col for col in dataset.columns if (col != 'target')]

active_index = 0
hist_sick = [(create_bar('sick', sicks, col, False) if i != active_index
           else create_bar('sick', sicks, col, True))
             for i, col in enumerate(hist_features)
            ]
hist_healthy = [(create_bar('healthy', healthy, col, False) if i != active_index
           else create_bar('healthy', healthy, col, True))
             for i, col in enumerate(hist_features)
            ]

total_data = hist_sick + hist_healthy
number_of_features = len(hist_features)
steps = []

for i in range(number_of_features):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * number_of_features],
        label = hist_features[i],
    )
    step['args'][1][i] = True
    steps.append(step)
    
sliders = [dict(
    active = active_index,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    autosize=True,
    yaxis=dict(
        title='valori',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=total_data, layout=layout)
py.iplot(fig, filename='bar_slider')

Come possiamo notare dal grafico soprastante, valori come *age*, *trestbs* e *chol* non sono troppo determinanti per la classificazione. Altri attributi come *cp*, *thal* e *slope* hanno valori tutto sommato ben distinti tra le due classi: questi saranno gli attributi che saranno più importanti nella fase di classificazione.

Una matrice di correlazione è una tabella che mostra i coefficienti di correlazione tra insiemi di variabili. 
Ogni variabile casuale ($X_i$) nella tabella è correlata con ciascuno degli altri valori nella tabella ($X_j$); questo permette di vedere quali coppie hanno la più alta correlazione.
La correlazione si riferisce a qualsiasi associazione statistica, ma nell'uso comune del termine si indica quanto due variabili siano vicine ad avere una relazione lineare l'una con l'altra.

In questo frangente useremo la correlazione di Pearson, che è una misura della correlazione lineare tra due variabili $X$ e $Y$. Secondo la diseguaglianza di Cauchy-Schwartz, tale valore è compreso tra +1 e -1, dove +1 è la correlazione positiva totale, 0 è nessuna correlazione lineare e -1 è una correlazione negativa totale.

Questo coefficiente è calcolato come $ \frac{cov(X, Y)}{\sigma_X\sigma_Y} $ dove:
* $cov$ si chiama covarianza ed è calcolata come ($\mu$ è il valore medio)  $$ cov(X, Y) = E[(X - \mu_X)(Y - \mu_Y)] $$
* $\sigma_X$ è la deviazione standard di $X$ $$ \sigma_X = \sqrt{E[X^2] - (E[X])^2} $$
* $\sigma_Y$ è la deviazione standard di $Y$ $$ \sigma_Y = \sqrt{E[Y^2] - (E[Y])^2} $$

In [22]:
def plot_correlation_matrix(matrix, title):

    z_text = np.around(matrix.values.tolist(), decimals=2)

    figure = ff.create_annotated_heatmap(z=matrix.values, 
                                         x=matrix.columns.tolist(), 
                                         y=matrix.index.tolist(),
                                         annotation_text=z_text,
                                         colorscale=COLORSCALE_HEATMAP,
                                         showscale=True)

    figure.layout.title = title
    figure.layout.autosize = False
    figure.layout.width = 850
    figure.layout.height = 850
    figure.layout.margin = go.layout.Margin(l=140, r=100, b=200, t=80)
    figure.layout.xaxis.update(side='bottom')
    figure.layout.yaxis.update(side='left')

    for i in range(len(figure.layout.annotations)):
        figure.layout.annotations[i].font.size = 8
                                    
    return py.iplot(figure, filename=title)

def plot_correlation_row(matrix, title):
    
    matrix = pd.Series.to_frame(matrix.loc['class']).transpose()
    z_text = np.around(matrix.values.tolist(), decimals=2)

    figure = ff.create_annotated_heatmap(z=matrix.values, 
                                         x=matrix.columns.tolist(), 
                                         y=matrix.index.tolist(),
                                         annotation_text=z_text,
                                         colorscale=COLORSCALE_HEATMAP,
                                         showscale=False)

    figure.layout.title = title
    figure.layout.autosize = False
    figure.layout.width = 850
    figure.layout.height = 220
    figure.layout.xaxis.update(side='bottom')
    figure.layout.yaxis.update(side='left')

    for i in range(len(figure.layout.annotations)):
        figure.layout.annotations[i].font.size = 8


    return py.iplot(figure, filename=title)


In [23]:
corr_dataset = dataset.corr(method='pearson')
plot_correlation_matrix(corr_dataset, 'Matrice di correlazione')

Ma un momento, questo tipo di grafico ci da tutte le informazioni che effettivamente ci servono? In realtà alcune variabili numeriche definiscono dei valori che sono di tipo categorico, per cui utilizzare un'analisi geometrica come quella di Pearson ha poco senso. 

Per arrivare ad ottenere un risultato più consistente, abbiamo quindi bisogno di qualcosa che sia simile alla correlazione, ma che funzioni anche per le varibili di tipo categorico: necessitiamo di una *misura di associazione* che non sia però simmetrica, perchè in caso contrario corriamo il rischio di perdere informazioni come ad esempio *l'attributo x implica la label y ma la label y non implica la feature x*.

**Theil's U**, anche conosciuta come **Uncertanty Coefficient** si adatta perfettamente alla nostra situazione. E' basato sull'entropia condizionale tra le variabili x e y; in altre parole, dato un valore X, esso mostra quanti stati ha y, e quanto spesso essi occorrono. L'input è del tipo $[0, 1]$ ed è asimmetrico (ossia $U(x, y) \neq U(y, x)$ ).

Il coefficiente è definito come: 

$$ U(X|Y) = \frac{H(X) - H(X|Y)}{H(X)} $$

dove:
* $ H(X) = \sum_x P_X(x)log(P_X(x)) $
* $ H(X|Y) = - \sum_{x,y} P_{X,Y}(x,y)log(P_{X,Y}(x,y)) $

In [24]:
def conditional_entropy(x, y):
    # entropy of x given y
    y_counter = Counter(y)
    xy_counter = Counter(list(zip(x,y)))
    total_occurrences = sum(y_counter.values())
    entropy = 0
    for xy in xy_counter.keys():
        p_xy = xy_counter[xy] / total_occurrences
        p_y = y_counter[xy[1]] / total_occurrences
        entropy += p_xy * math.log(p_y/p_xy)
    return entropy

def theil_u(x, y):
    s_xy = conditional_entropy(x, y)
    x_counter = Counter(x)
    total_occurrences = sum(x_counter.values())
    p_x = list(map(lambda n: n/total_occurrences, x_counter.values()))
    s_x = ss.entropy(p_x)
    if s_x == 0:
        return 1
    else:
        return (s_x - s_xy) / s_x

def create_theil_matrix(data):

    columns = data.columns
    matrix = []
    for col in columns:
        row = []
        for j in range(0, len(columns)):
            u = np.around(theil_u(data[col].tolist(), data[columns[j]].tolist()), decimals=4)
            row.append(u)
        matrix.append(row)
    
    matrix = np.array(matrix)

    matrix = pd.DataFrame(data=matrix,
                 columns=columns,
                 index=columns)
    return matrix


In [25]:
corr_dataset = create_theil_matrix(dataset)
plot_correlation_matrix(corr_dataset, "Theil_u correlation matrix")

In [59]:
feature_histogram(dataset, 'chol', 'distribuzione chol')

In [29]:
names = dataset.columns
inverse_correlation = 1 - corr_dataset # This is the 'dissimilarity' method

fig = ff.create_dendrogram(inverse_correlation, 
                           labels=names, 
                           colorscale=COLOR_PALETTE, 
                           linkagefun=lambda x: hc.linkage(x, 'average'))

fig['layout'].update(dict(
    title="Dendogramma di correlazione tra gli attributi",
    width=800, 
    height=600,
    xaxis=dict(
        title='Features',
    ),
    yaxis=dict(
        title='Distance',
    ),
))
iplot(fig, filename='dendrogram_corr_clustering')

### NORMALIZZAZIONE DEI DATI

La maggior parte delle volte, i set di dati contengono caratteristiche molto variabili in termini di grandezze, unità di misura e range. 
Poichè la maggior parte degli algoritmi di machine learning utilizza la distanza euclidea per misurare la distanza tra due dati nei loro calcoli, questo potrebbe causare dei problemi.

Se lasciati così come sono, questi algoritmi prendono solo la grandezza delle funzioni che trascurano le unità: i risultati variano molto tra le diverse unità, ad esempio tra 5Kg e 5000g.

Per questo, le caratteristiche con grandezze elevate peseranno molto di più nei calcoli rispetto agli attributi con minor grandezza. Per risolvere questo problema, dobbiamo portare tutti gli attributi sullo stesso livello di grandezza e ciò si può fare con il ridimensionamento.

Per fare ciò, useremo lo StandardScaler, che standardizza i nostri dati sia con la media che con la deviazione standard. L'operazione matematica eseguita sarà: $$ x' = \frac{x - \mu_x}{\sigma_x} $$
Dopodichè, spezzeremo il nostro dataset in un array di dati non classificati e un array di label, che utilizzeremo nella fase di classificazione.

In [7]:
def dataframe_to_array(data):
    X_data = data.drop(['target'], axis=1)
    y_data = data['target']
    return X_data, y_data

def scale_data(X_data):
    scaler = StandardScaler(with_mean=True, with_std=True, copy=True)
    return scaler.fit_transform(X_data)

X_data, y_data = dataframe_to_array(dataset)
X_scaled_data = scale_data(X_data)

### PRINCIPAL COMPONENT ANALYSIS

Quando i nostri dati sono rappresentati da una matrice troppo grande (ossia il numero di dimensioni è troppo alto) risulta difficile estrarre le caratteristiche più interessanti e trovare correlazioni tra di loro, senza considerare che lo spazio occupato è più alto del necessario.
La PCA è una tecnica che consente di ridurre la dimensionalità dei dati preservando le differenze più importanti che interccorrono tra i campioni.

Questa trasformazione è definiti in modo tale che la prima componente principale abbia la varianza più grande possibile (ovvero, tiene conto il più possibile della variabilità dei valori assunti da una variabile) e ciascun componente successivo ha a sua volta la varianza più alta possibile sotto il vincolo di essere ortogonale alle componenti precedenti.
I vettori risultanti (essendo ciascuno una combinazione lineare delle variabili e contenente N osservazioni) costituiscono una base ortogonale.

In [14]:
def plot_cumulative_variance(pca, title):
    
    """
    Grafico della varianza cumulativa di tutte le PC
    """   

    tot_var = np.sum(pca.explained_variance_)
    ex_var = [(i / tot_var) * 100 for i in sorted(pca.explained_variance_, reverse=True)]
    cum_ex_var = np.cumsum(ex_var)

    cum_var_bar = go.Bar(
        x=list(range(1, len(cum_ex_var) + 1)), 
        y=ex_var,
        name="Varianza di ogni componente",
        marker=dict(
            color=PLOTLY_COLORS[0],
        ),
        opacity=PLOTLY_OPACITY
        )

    variance_line = go.Scatter(
        x=list(range(1, len(cum_ex_var) + 1)),
        y=cum_ex_var,
        mode='lines+markers',
        name="Varianza cumulativa",
        marker=dict(
            color=PLOTLY_COLORS[1],
        ),
        opacity=PLOTLY_OPACITY,
        line=dict(
            shape='hv',
        ))
    data = [cum_var_bar, variance_line]
    layout = go.Layout(
        autosize=True,
        title=title,
        yaxis=dict(
            title='Varianza (%)',
        ),
        xaxis=dict(
            title="Componenti principali",
            dtick=1,
            rangemode='nonnegative'
        ),
        legend=dict(
            x=0,
            y=1,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)

def compress_data(X_dataset, n_components, plot_comp=False, graph_title=''):
    
    """
    Performs pca reduction of a dataset.

    :param (array of arrays) X_dataset: Dataset to reduce
    :param (int) n_components: N components to project on 
    :param (bool) plot_comp: Plot explained variance

    :returns (pandas dataframe) X_df_reduced: pandas dataframe with reduced dataset
    :return (iplot) p: Plot, only if plot_com equals True. To plot the graph,
                       simply call the return value.
    """   

    pca = PCA(random_state=11) #random seed
    projected_data = pca.fit_transform(X_dataset)

    if plot_comp:
        p = plot_cumulative_variance(pca, graph_title)

    pca.components_ = pca.components_[:n_components]
    reduced_data = np.dot(projected_data, pca.components_.T)
    X_df_reduced = pd.DataFrame(reduced_data, columns=["PC#%d" % (x + 1) for x in range(n_components)])
    if plot_comp:
        return p, X_df_reduced
    else:
        return X_df_reduced
    

In [15]:
plot, X_df_ohc_reduced = compress_data(X_dataset=X_scaled_data,
              n_components=13,
              plot_comp=True,
              graph_title="Varianza individuale e cumulativa")
plot

In [53]:
X_df_ohc_reduced.head(4)

,PC#1,PC#2,PC#3,PC#4,PC#5,PC#6,PC#7,PC#8,PC#9,PC#10,PC#11,PC#12,PC#13
0,0.674659,2.095604,3.809485,0.111267,1.151868,-1.089498,0.224776,-0.555006,-0.364659,1.480557,-0.116801,-0.193078,0.089707
1,0.732016,1.850302,-0.558395,-0.096317,2.620567,-1.278350,0.989618,-0.101235,1.425391,1.339730,1.428196,0.094293,0.304661
2,-0.523563,-0.160394,-0.290459,-0.790662,-0.092465,1.432148,1.631711,0.391299,0.744643,1.034625,0.638197,-1.221829,0.216461
3,0.181329,-0.676779,0.040449,0.579549,0.353748,-0.006323,-0.480461,0.870860,1.521938,-0.195859,0.614323,-1.021186,-0.185181


Cercheremo ora di utilizzare uno scatter-plot per mostrare se un algoritmo di clustering applicato sulle prime due componenti principali è in grado di separare i campioni in due cluster diversi.

PCA cerca di trovare combinazioni di features che conducono alla massima separazione tra i dati; ciò significa che, se avessimo una dimensione nel nostro dataset che rimane la stessa per tutti i dati, questa non verrebbe considerata, da solo o come combinazione, tra le componenti principali. Solo le caratteristiche che variano molto da dato a dato fanno parte di esse. Di conseguenza, i punti dovrebbero apparire abbastanza distanti l'uno dall'altro sul grafico.

Il grafico potrebbe avere anche poco senso, e ciò può avvenire per due fattori principali 
1. E' presente molta varianza nel dataset, e perciò le prime due componenti non bastano a rappresentare significativamente i dati
2. L'algoritmo di clustering si focalizza su features che non sono considerate importanti da PCA

In [23]:
values = X_scaled_data
pca = PCA(n_components=2)
x = pca.fit_transform(values)

kmeans = KMeans(n_clusters=2, random_state=11)
X_clustered = kmeans.fit_predict(values)

c1_idx = np.where(X_clustered == 0)
c2_idx = np.where(X_clustered == 1)

p1 = go.Scatter(
    x=np.take(x[:,0], indices=c1_idx)[0],
    y=np.take(x[:,1], indices=c1_idx)[0],
    mode='markers',
    name="Cluster1",
    marker=dict(
        color=COLOR_PALETTE[2],
    ),
    opacity=PLOTLY_OPACITY)

p2 = go.Scatter(
    x=np.take(x[:,0], indices=c2_idx)[0],
    y=np.take(x[:,1], indices=c2_idx)[0],
    mode='markers',
    name="Cluster2",
    marker=dict(
        color=COLOR_PALETTE[5],
    ),
    opacity=PLOTLY_OPACITY)

data = [p1, p2]

layout = go.Layout(
    title='Dati clusterizzati usando le prime due componenti',
    autosize=True,
    yaxis=dict(
        title='Seconda componente',
    ),
    xaxis=dict(
        title="Prima componente",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='clusters-scatter')

### DATI NON CATEGORICI

Partiamo con l'analizzare i dati di tipo non categoriche.

In [30]:
#f,ax=plt.subplots(3,2,figsize=(12,12))
#f.delaxes(ax[2,1])
#
#numerical_features = ['age','thalach','chol','trestbps','oldpeak']
#
#for i,feature in enumerate(numerical_features):
#    sns.distplot(dataset[feature], ax=ax[i//2,i%2], kde_kws={"color":"white"}, hist=False )
#
#    # Get the two lines from the ax[i//2,i%2]es to generate shading
#    l1 = ax[i//2,i%2].lines[0]
#
#    # Get the xy data from the lines so that we can shade
#    x1 = l1.get_xydata()[:,0]
#    y1 = l1.get_xydata()[:,1]
#    ax[i//2,i%2].fill_between(x1,y1, color="#4791C5", alpha=0.8)
#
#    #grid
#    #ax[i//2,i%2].grid(b=True, which='major', color='grey', linewidth=0.3)
#    
#    ax[i//2,i%2].set_title('Distribution of {}'.format(feature), fontsize=18)
#    ax[i//2,i%2].set_ylabel('count', fontsize=12)
#    ax[i//2,i%2].set_xlabel('Modality', fontsize=12)
#
#    #sns.despine(ax[i//2,i%2]=ax[i//2,i%2], left=True)
#    ax[i//2,i%2].set_ylabel("frequency", fontsize=12)
#    ax[i//2,i%2].set_xlabel(str(feature), fontsize=12)
#    
#plt.tight_layout()
#plt.show()


In [2]:
#numerical_features = ['age','thalach','chol','trestbps','oldpeak']
##hist_features = [col for col in dataset.columns if col in numerical_features]
#
#steps = []
#active_index = 0
#
#number_of_features = len(numerical_features)
#
#data = [(create_distplot(numerical_features[i], dataset[numerical_features[i]], col, False) if i!=active_index
#         else create_distplot(numerical_features[i], dataset[numerical_features[i]], col, True))
#        for i, col in zip(range(number_of_features), enumerate(numerical_features))]
#
#for i in range(number_of_features):
#    step = dict(
#        method = 'restyle',
#        args = ['visible', [False] * number_of_features],
#        label = numerical_features[i],
#    )
#    step['args'][1][i] = True
#    steps.append(step)
#    
#sliders = [dict(
#    active = active_index,
#    currentvalue = dict(
#        prefix = 'Feature: ',
#        xanchor = 'center',
#    ),
#    pad = {"t": 50},
#    steps = steps,
#    len = 1,
#)]
#    
#layout = dict(
#    sliders = sliders,
#    autosize = True,
#    yaxis = dict(
#        title = 'Frequency',
#        automargin = True,
#    ),
#    barmode = 'group',
#    bargap = 0.15,
#    bargroupgap = 0.1
#)
#    
#fig = dict(data=data, layout=layout)
#py.iplot(fig, filename='numerical_slider')

### DATI CATEGORICI

In [31]:
#f,ax=plt.subplots(4,2,figsize=(12,12))
#
#for i,feature in enumerate(['sex','cp','fbs','restecg','exang','slope','ca','thal']):
#    sns.countplot(x=feature,data=dataset,ax=ax[i//2,i%2], palette = 'Blues_d', alpha=0.8, edgecolor=('white'), linewidth=2)
#    ax[i//2,i%2].set_title('Count of {}'.format(feature), fontsize=18)
#    ax[i//2,i%2].set_ylabel('count', fontsize=12)
#    ax[i//2,i%2].set_xlabel('modality', fontsize=12)
#
#
#plt.tight_layout()
#plt.show()

### CLASSIFICAZIONE

In questa fase esploreremo diversi metodi di supervised learning e vedremo alla fine quale di questi classifica meglio i nostri dati.

Prima di iniziare, vediamo quale tipo di pre-processed data è meglio utilizzare. Siccome il nostro dataset è particolarmente piccolo, sicuramente la riduzione della dimensionalità vista poco sopra con PCA non è necessaria.

La prima cosa da fare è splittare i dati:

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled_data,y_data, test_size=0.2, random_state=RANDOM_SEED, stratify=y_data)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_df_ohc_reduced,y_data, test_size=0.2, random_state=RANDOM_SEED, stratify=y_data)

In [97]:
def print_gridcv_scores(grid_search, n=5):
    """
    Prints the best score achieved by a grid_search, alongside with its parametes

    :param (estimator) clf: Classifier object
    :param (int) n: Best n scores 
    """    
    
    t = PrettyTable()

    print("Best grid scores on validation set:")
    indexes = np.argsort(grid_search.cv_results_['mean_test_score'])[::-1][:n]
    means = grid_search.cv_results_['mean_test_score'][indexes]
    stds = grid_search.cv_results_['std_test_score'][indexes]
    params = np.array(grid_search.cv_results_['params'])[indexes]
    
    t.field_names = ['Score'] + [f for f in params[0].keys()] 
    for mean, std, params in zip(means, stds, params):
        row=["%0.3f (+/-%0.03f)" % (mean, std * 2)] + [p for p in params.values()]
        t.add_row(row)
    print(t)
               

def param_tune_grid_cv(clf, params, X_train, y_train, cv, execution_time=False):
    """
    Function that performs a grid search over some parameters

    :param (estimator) clf: Classifier object
    :param (dictionary) params: parameters to be tested in grid search
    :param (array-like) X_train: List of data to be trained with
    :param (array-like) y_train: Target relative to X for classification or regression
    :param (cross-validation generator) cv: Determines the cross-validation splitting strategy
    """ 
    if execution_time:
      start = time.perf_counter()
    pipeline = Pipeline([('clf', clf)])
    grid_search = GridSearchCV(estimator=pipeline, 
                               param_grid=params, 
                               cv=cv, 
                               n_jobs=-1,       # Use all processors
                               scoring='f1',    # Use f1 metric for evaluation
                               return_train_score=True)
    grid_search.fit(X_train, y_train)
    if execution_time:
      end = time.perf_counter()
      return grid_search, "%.4f" % (end-start)
    return grid_search
   

def score(clfs, datasets):
    """
    Function that scores a classifier on some data
    
    :param (array of estimator) clf: Array of classifiers
    :param (dictionary) params: Dictionary of test data, passed like [(X_test, y_test)]

    """  
    scores = []
    for c, (X_test, y_test) in zip(clfs, datasets):
        scores.append(c.score(X_test, y_test))

    return scores


def hexToRGBA(hex, alpha):

    """
    Function that returns an rgba value from an hex and an opacity value
    
    :param (String) clf: Hex value 
    :param (float) params: Value between 0 and 1 indicating opacity

    """  

    r = int(hex[1:3], 16)
    g = int(hex[3:5], 16)
    b = int(hex[5:], 16)

    if alpha:
        return "rgba(" + str(r) + ", " + str(g) + ", " + str(b) + ", " + str(alpha) + ")"
    else:
        return "rgb(" + str(r) + ", " + str(g) + ", " + str(b) + ")"


def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(.008, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, 
                                                            X, 
                                                            y, 
                                                            cv=cv, 
                                                            n_jobs=n_jobs, 
                                                            train_sizes=train_sizes, 
                                                            scoring="f1", 
                                                            random_state=RANDOM_SEED,
                                                           )
    
    print(train_sizes, train_scores, test_scores)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    # Prints lower bound (mean - std) of train 
    trace1 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean - train_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[0], 0.4),
        ),
    )
    # Prints upper bound (mean + std) of train
    trace2 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean + train_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[0], 0.4),
        ),
    )
    
    # Prints mean train score line
    trace3 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean, 
        showlegend=True,
        name="Train score",
        line = dict(
            color = PLOTLY_COLORS[0],
        ),
    )
    
    # Prints lower bound (mean - std) of test 
    trace4 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean - test_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[1], 0.4),
        ),
    )
        # Prints upper bound (mean + std) of test
    trace5 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean + test_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = hexToRGBA(PLOTLY_COLORS[1], 0.4),
        ),
    )

    # Prints mean test score line 
    trace6 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean, 
        showlegend=True,
        name="Test score",
        line = dict(
            color = PLOTLY_COLORS[1],
        ),
    )
    
    data = [trace1, trace2, trace3, trace4, trace5, trace6]
    layout = go.Layout(
        title=title,
        autosize=True,
        yaxis=dict(
            title='F1 Score',
        ),
        xaxis=dict(
            title="#Training samples",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)


def print_confusion_matrix(gs, X_test, y_test):

    """
    Function that prints confusion matrix for a classifier
    
    :param (estimator) clf: Classifier object
    :param (array-like) X_test: List of data to be tested with
    :param (array-like) y_test: List of labels for test 
    """  

    gs_score = gs.score(X_test, y_test)
    y_pred = gs.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)
    t = PrettyTable()
    t.add_row(["True Sick", cm[0][0], cm[0][1]])
    t.add_row(["True Healthy", cm[1][0], cm[1][1]])
    t.field_names = [" ", "Predicted Sick", "Predicted Healthy"]
    print(t)

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix
    cm_df = pd.DataFrame(cm.round(3), index=["True Sick", "True Healthy"], columns=["Predicted sick", "Predicted healthy"])
    cm_df


def print_raw_score(clf, X_test, y_test):
    """
    Function that scores a classifier on some data
    
    :param (array of estimator) clf: Array of classifiers
    :param (array-like) X_test: List of data to be tested with
    :param (array-like) y_test: List of labels for test 

    """  
    print("Score achieved by NB: %0.3f" % (score([clf], [(X_test, y_test)])[0]))


def plot_feature_importance(feature_importance, title):
    """
    Function that plots feature importance for a decision tree or a random forest classifier
    
    :param (dictionary) feature_importance: Dictionary of most important features sorted
    :param (str) title: Title of the plot

    """ 
    
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = PLOTLY_COLORS[0]),
        opacity=PLOTLY_OPACITY,
        name='Feature importance'
    )
    data = [trace1]
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='feature importance',
            automargin=True,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)


def print_performances(classifiers, classifier_names, auc_scores, X_test, y_test):
  
    """
    Function that scores a classifier on some data
    
    :param (array of estimator) clf: Array of classifiers
    :param (array-like) classifier_names: Title of the classifier
    :param (array-like) auc-score: Auc scores
    :param (array-like) X_test: List of data to be tested with
    :param (array-like) y_test: List of labels for test 

    """ 

    accs = []
    recalls = []
    precision = []
    results_table = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1", "auc"])
    for (i, clf), name, auc in zip(enumerate(classifiers), classifier_names, auc_scores):
        y_pred = clf.predict(X_test)
        row = []
        row.append(accuracy_score(y_test, y_pred))
        row.append(precision_score(y_test, y_pred))
        row.append(recall_score(y_test, y_pred))
        row.append(f1_score(y_test, y_pred))
        row.append(auc)
        row = ["%.3f" % r for r in row]
        results_table.loc[name] = row
    return results_table



In [35]:
kf = StratifiedKFold(n_splits=5, random_state=RANDOM_SEED)
clf_nb = GaussianNB()
clf_knn = KNeighborsClassifier()
clf_rf = RandomForestClassifier(random_state=RANDOM_SEED)
clf_lr = LogisticRegression(random_state=RANDOM_SEED)
clf_svm = SVC(random_state=RANDOM_SEED)

clfs = [clf_nb, clf_knn, clf_rf, clf_lr, clf_svm]
clfs_ng = [clf_nb, clf_rf]
TEST_PARAMS = {}

In [36]:
all_test_results = []
all_gss = []
times = np.zeros(2)
t = 0

for clf in clfs:
    gs_ohc_pc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train, y_train, kf, execution_time=True)
    times[0] = times[0] + float(t)

    gs_ohc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pca, y_train_pca, kf, execution_time=True)
    times[1] = times[1] + float(t)

    gss = [gs_ohc_pc, gs_ohc]
    all_gss.append(gss)
    test_results = score(gss, [(X_test, y_test),
                                (X_test_pca, y_test_pca)])
    all_test_results.append(test_results)

all_test_results_ng = []
all_gss_ng = []
times_ng = np.zeros(3)

for clf in clfs_ng:
    gs_full, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train, y_train, kf, execution_time=True)
    times_ng[0] = times_ng[0] + float(t)
  
    #gs_pc, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pc, y_train_pc, kf, execution_time=True)
    #times[1] = times[1] + float(t)
  
    gs_drop, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pca, y_train_pca, kf, execution_time=True)
    times_ng[1] = times_ng[1] + float(t)
  
    #gs_pc_drop, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train_pc_drop, y_train_pc_drop, kf, execution_time=True)
    #times[3] = times[3] + float(t)
  
    #gs_no_stalk, t = param_tune_grid_cv(clf, TEST_PARAMS, X_train, y_train, kf, execution_time=True)
    #times_ng[2] = times_ng[2] + float(t)

    gss_ng = [gs_full, gs_drop, gs_no_stalk]
    all_gss_ng.append(gss_ng)
    test_results = score(gss_ng, [(X_test, y_test),
                                  (X_test_pca, y_test_pca)
                                 ])
    all_test_results_ng.append(test_results)


In [44]:
def print_results(column_names, row_names, values):
    """
    Function that prints a table using prettytable library
    
    :param (array) column_names: Array of column names
    :param (array) row_names: Array of row names
    :param (matrix) values: Values of the table

    """ 
    t = PrettyTable()
    t.field_names = column_names
    
    all_rows = []
    result_row = []

    for name, results in zip(row_names, values):
        result_row.append(name)
        for r in results:
            result_row.append("%.3f" % r)
        all_rows.append(result_row)
        result_row = []

    all_rows = sorted(all_rows, key=lambda kv: kv[1], reverse=True)
    for k in all_rows:
        t.add_row(k)
    
    print(t)


dataset_strings = [" ",
                   "dataset with scaled data",
                   "dataset with PCA"]

dataset_strings_ng = [" ", "full dataset",
                      "dataset with dropped missing values"]

row_names = ["Naive Bayes", "KNN", "Random Forest", "Logistic Regression", "SVM"]
#row_names_ng = ["Naive Bayes", "Random Forest"]

print_results(dataset_strings, row_names, all_test_results)
#print_results(dataset_strings_ng, row_names_ng, all_test_results_ng)

+---------------------+--------------------------+------------------+
|                     | dataset with scaled data | dataset with PCA |
+---------------------+--------------------------+------------------+
|    Random Forest    |          0.866           |      0.727       |
| Logistic Regression |          0.853           |      0.853       |
|         SVM         |          0.829           |      0.829       |
|         KNN         |          0.800           |      0.800       |
|     Naive Bayes     |          0.776           |      0.754       |
+---------------------+--------------------------+------------------+


In [40]:
print_confusion_matrix(all_gss_ng[0][np.argmin(all_test_results_ng[0])], X_test_pca, y_test_pca)

+--------------+----------------+-------------------+
|              | Predicted Sick | Predicted Healthy |
+--------------+----------------+-------------------+
|  True Sick   |       18       |         10        |
| True Healthy |       7        |         26        |
+--------------+----------------+-------------------+


In [39]:
means = np.mean(all_test_results, axis=0)
row_names = ["Total train time (s)", "Mean score for dataset"]
print_results(dataset_strings, row_names, [times, means])

+------------------------+--------------------------+------------------+
|                        | dataset with scaled data | dataset with PCA |
+------------------------+--------------------------+------------------+
|  Total train time (s)  |          4.714           |      2.412       |
| Mean score for dataset |          0.825           |      0.792       |
+------------------------+--------------------------+------------------+


In [48]:
print("The dataset that gives the best overall performances is:")
print("\t- " + dataset_strings[means.argmax()+1] + ", with a score of " + str("%.3f" % means.max()))

The dataset that gives the best overall performances is:
	- dataset with scaled data, with a score of 0.825


### LOGISTIC REGRESSION

Il primo classificatore che andiamo ad analizzare è la logistic regression: esso usa la funzione di Sigmoid per classificare i nostri sample:
* $ P(y=0 | X;\theta) = g(w^TX) = \frac{1}{1+e^{w^TX}} $
* $ P(y=1 | X;\theta) = 1 - g(w^TX) = \frac{e^{w^TX}}{1+e^{w^TX}} $

Questo modello, rispetto alla linear regression, può modellare meglio la zona compresa tra 0 e 1. Per conoscere i pesi, bisogna calcolare la MLE (Maximum Likelihood Estimation, ossia scegliere un valore che massimizzi la probabilità dei dati osservati) ed applicare l'algoritmo di gradient descent fino alla convergenza del valore di accuratezza.

Essi sono:
* **liblinear**: Solver, migliore per dataset più piccoli.
* **C**: "forza di regolarizzazione", valore compreso tra 0,01 e 100. Più è piccolo il valore, maggiore è la regolarizzazione
* **penalty**: penalità "l1" e "l2" per la regolarizzazione, che sono definite come:
    * l1, che penalizza ogni errore allo stesso modo $ S = \sum_{i=1}^{n}| y_i - f(x_i) | $
    * l2, che penalizza maggiormente valori più grandi $ S = \sum_{i=1}^{n}(y_i - f(x_i))^2 $

Dove $y_i$ è la vera label mentre $f(x_i)$ è quella assegnata.    

In [86]:
clf_lr = LogisticRegression(random_state=RANDOM_SEED)
gs_pc_lr = param_tune_grid_cv(clf_lr, LOGISTIC_REGRESSION_PARAMS, X_train, y_train, kf)
print_gridcv_scores(gs_pc_lr, n=5)

Best grid scores on validation set:
+------------------+--------+--------------+-------------+
|      Score       | clf__C | clf__penalty | clf__solver |
+------------------+--------+--------------+-------------+
| 0.868 (+/-0.032) |  0.1   |      l2      |    lbfgs    |
| 0.868 (+/-0.032) |  0.1   |      l2      |  newton-cg  |
| 0.867 (+/-0.031) |  0.1   |      l2      |  liblinear  |
| 0.865 (+/-0.014) | 0.001  |      l2      |  liblinear  |
| 0.863 (+/-0.010) |  0.01  |      l2      |  liblinear  |
+------------------+--------+--------------+-------------+


In [98]:
plot_learning_curve(gs_pc_lr.best_estimator_, "Learning curve of Logistic Regression", 
                    X_train,
                    y_train,
                    cv=5,
                   )

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1